In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
import glob
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, MultiHeadAttention, GlobalAveragePooling1D, LayerNormalization

os.chdir("drive/MyDrive/Cyber_paper") #to get into DAKOTA folder, comment out if already in there
current_dir = os.getcwd()
data_dir = current_dir + '/DATA'
print("Current Data Dir: ", data_dir)
all_sess = glob.glob("DATA/*")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
n = 1
sessions = all_sess[:]
tl = sessions[n]
del sessions[n]
df = pd.read_csv(glob.glob(f"{tl}/merged_features_all.csv")[0])
df["LABEL"] = 1
df.head()

In [ ]:
import matplotlib.pyplot as plt

figure, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 8))

axes[0].plot(df[df.columns[0]], linewidth=0.5, zorder=1, label=df.columns[0])
axes[0].plot(df[df.columns[8]], linewidth=0.5, zorder=1, label=df.columns[8])
axes[0].plot(df[df.columns[16]], linewidth=0.5, zorder=1, label=df.columns[16])
axes[0].legend()
axes[0].set_title('Accelerometer')

axes[1].plot(df[df.columns[24]], linewidth=0.5, zorder=1, label=df.columns[24])
axes[1].plot(df[df.columns[32]], linewidth=0.5, zorder=1, label=df.columns[32])
axes[1].plot(df[df.columns[40]], linewidth=0.5, zorder=1, label=df.columns[40])
axes[1].legend()
axes[1].set_title('Magnetometer')

axes[2].plot(df[df.columns[48]], linewidth=0.5, zorder=1, label=df.columns[48])
axes[2].plot(df[df.columns[56]], linewidth=0.5, zorder=1, label=df.columns[56])
axes[2].plot(df[df.columns[64]], linewidth=0.5, zorder=1, label=df.columns[64])
axes[2].legend()
axes[2].set_title('Gyroscope')

plt.tight_layout()
plt.show()


In [ ]:
df.shape

In [ ]:

df2 = pd.concat((pd.read_csv(glob.glob(f"{tl}/merged_features_all.csv")[0]) for tl in sessions))
df2["LABEL"] = 0
df2.head()

In [ ]:

merged_df = pd.concat((df, df2))

columns = merged_df.columns

scaler = MinMaxScaler()
model = scaler.fit(merged_df)
scaled_data = pd.DataFrame(model.transform(merged_df), columns= columns)
print(scaled_data.shape)
scaled_data["LABEL"].value_counts()

In [ ]:
scaled_data.head()

In [ ]:

# print(scaled_data)

train, validate, test = \
              np.split(scaled_data.sample(frac=1, random_state=42),
                       [int(.88*len(scaled_data)), int(.9*len(scaled_data))])

X_train_, y_train_ = train.drop("LABEL", axis=1), train["LABEL"]
X_test, y_test = test.drop("LABEL", axis=1), test["LABEL"]
X_val, y_val = validate.drop("LABEL", axis=1), validate["LABEL"]

# print(X_train.shape, y_train.shape)
# print(X_test.shape, y_test.shape)
# print(X_val.shape, y_val.shape)


In [ ]:
from imblearn.over_sampling import ADASYN
sm = ADASYN(sampling_strategy = 0.7, n_neighbors = 6)
# print(len(X_train))
X_train, y_train = sm.fit_resample(X_train_, y_train_)
print(X_train.shape, y_train.shape)

In [ ]:
from imblearn.over_sampling import SVMSMOTE
sm = SVMSMOTE(sampling_strategy = .7, k_neighbors = 10)
# print(len(X_train))
X_train, y_train = sm.fit_resample(X_train_, y_train_)
print(X_train.shape, y_train.shape)

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy = 0.7, k_neighbors = 5)
# print(len(X_train))
X_train, y_train = sm.fit_resample(X_train_, y_train_)
print(X_train.shape, y_train.shape)

In [ ]:
y_train.value_counts()

In [ ]:
X_train = np.resize(X_train, (X_train.shape[0], 126, 1))
X_test = np.resize(X_test, (X_test.shape[0], 126, 1))
print(X_train.shape)

In [ ]:
!pip install joblib==1.2.0

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_len, d_model):
        super(PositionalEncoding, self).__init__()
        self.positional_encoding = self.get_positional_encoding(max_len, d_model)

    def get_positional_encoding(self, max_len, d_model):
        pos = np.arange(max_len)[:, np.newaxis]
        i = np.arange(d_model)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        angle_rads = pos * angle_rates
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, inputs):
        return inputs + self.positional_encoding[:, :tf.shape(inputs)[1], :]


class ContAuth_trans:
    def __init__(self, hidden_units, dropout_rate, input_shape, num_classes, learning_rate, batch_size, early_stopping_threshold, alpha):
        self.hidden_units = hidden_units
        self.dropout_rate = dropout_rate
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.early_stopping_threshold = early_stopping_threshold
        self.alpha = alpha
        self.model = self.create_model()

    def create_model(self):
        inputs = Input(shape=self.input_shape)

        position_encoding = PositionalEncoding(self.input_shape[0], self.hidden_units)(inputs)

        x = Dense(self.hidden_units, activation='relu')(position_encoding)
        x = Dropout(self.dropout_rate)(x)

        attention = MultiHeadAttention(num_heads=4, key_dim=self.hidden_units)(x, x)
        attention = Dropout(self.dropout_rate)(attention)
        attention = LayerNormalization()(attention)

        attention = MultiHeadAttention(num_heads=8, key_dim=int(self.hidden_units/2))(attention, attention)
        attention = Dropout(self.dropout_rate)(attention)
        attention = LayerNormalization()(attention)

        x = GlobalAveragePooling1D()(attention)

        x = Dense(128, activation='relu')(x)
        x = Dropout(self.dropout_rate)(x)
        outputs = Dense(self.num_classes, activation='softmax')(x)

        model = Model(inputs=inputs, outputs=outputs)
        return model

    def compile_model(self):
        optimizer = Adam(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer,
                           loss='sparse_categorical_crossentropy',
                           metrics=['accuracy'])

    def train_model(self, X_train, y_train, X_val, y_val, epochs):
        early_stopping = EarlyStopping(monitor='val_loss',
                                       patience=20,
                                       min_delta=self.early_stopping_threshold,
                                       restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                      patience=2, min_lr=0.00001, mode='auto', verbose=1)
        checkpointer = ModelCheckpoint("saved_model/model_t2.keras", verbose=1, save_best_only=True)

        history = self.model.fit(X_train, y_train,
                                 validation_data=(X_val, y_val),
                                 epochs=epochs,
                                 batch_size=self.batch_size,
                                 callbacks=[early_stopping, reduce_lr, checkpointer])
        return history

    def extract_features(self, X):
        feature_extractor = Model(inputs=self.model.input, outputs=self.model.get_layer(index=-4).output)
        features = feature_extractor.predict(X)
        return features

    def train_sgd_classifier(self, X_train_features, y_train, X_val_features, y_val):
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_features)
        X_val_scaled = scaler.transform(X_val_features)

        sgd = SGDClassifier(loss='hinge', penalty='elasticnet', alpha=self.alpha, l1_ratio=0.5, max_iter=1000, tol=1e-3)
        sgd.fit(X_train_scaled, y_train)

        y_val_pred = sgd.predict(X_val_scaled)
        accuracy = accuracy_score(y_val, y_val_pred)
        return sgd, accuracy

    def data_incremental_training(self, X_new, y_new, X_test, y_test, epochs):
        history = self.train_model(X_new, y_new, X_test, y_test, epochs)

        X_train_features = self.extract_features(X_new)
        X_test_features = self.extract_features(X_test)

        sgd, accuracy = self.train_sgd_classifier(X_train_features, y_new, X_test_features, y_test)

        return history, accuracy

hidden_units = 256
dropout_rate = 0.5
input_shape = (126, 1)
num_classes = 2
learning_rate = 0.001
batch_size = 32
early_stopping_threshold = 0.00001
alpha = 0.01

try:
    del model
except:
    pass

model = ContAuth_trans(hidden_units, dropout_rate, input_shape, num_classes, learning_rate, batch_size, early_stopping_threshold, alpha)
model.compile_model()

# Print model summary
print(model.model.summary())


In [ ]:
class ContAuth:
    def __init__(self, hidden_units, dropout_rate, input_shape, num_classes, learning_rate, batch_size, early_stopping_threshold, alpha):
        self.hidden_units = hidden_units
        self.dropout_rate = dropout_rate
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.early_stopping_threshold = early_stopping_threshold
        self.alpha = alpha
        self.model = self.create_model()

    def create_model(self):
        inputs = Input(shape=self.input_shape)
        x = LSTM(self.hidden_units, return_sequences=False)(inputs)
        x = Dropout(self.dropout_rate)(x)
        x = Dense(int(self.hidden_units/2), activation='relu')(x)
        x = Dropout(self.dropout_rate)(x)
        outputs = Dense(self.num_classes, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=outputs)
        return model

    def compile_model(self):
        optimizer = Adam(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer,
                           loss='sparse_categorical_crossentropy',
                           metrics=['accuracy'])

    def train_model(self, X_train, y_train, X_val, y_val, epochs):
        if(epochs > 20): pat = 20
        else: pat = 2

        early_stopping = EarlyStopping(monitor='val_loss',
                                       patience=pat,
                                       min_delta=self.early_stopping_threshold,
                                       restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=2, min_lr=0.00001, mode='auto', verbose=1)
        checkpointer = ModelCheckpoint("saved_model/model_t2.keras", verbose=1, save_best_only=True)

        history = self.model.fit(X_train, y_train,
                                 validation_data=(X_val, y_val),
                                 epochs=epochs,
                                 batch_size=self.batch_size,
                                 callbacks=[early_stopping, reduce_lr, checkpointer])

        return history

    def extract_features(self, X):
        feature_extractor = Model(inputs=self.model.input, outputs=self.model.layers[-1].output)
        features = feature_extractor.predict(X)
        return features

    def train_sgd_classifier(self, X_train_features, y_train, X_val_features, y_val):
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_features)
        X_val_scaled = scaler.transform(X_val_features)

        sgd = SGDClassifier(loss='hinge', penalty='elasticnet', alpha=self.alpha, l1_ratio=0.5, max_iter=1000, tol=1e-3)
        sgd.fit(X_train_scaled, y_train)

        y_val_pred = sgd.predict(X_val_scaled)
        accuracy = accuracy_score(y_val, y_val_pred)
        return sgd, accuracy

    def data_incremental_training(self, X_new, y_new, X_test, y_test, epochs):
        X_train_features = self.extract_features(X_new)
        X_test_features = self.extract_features(X_test)

        sgd, accuracy = self.train_sgd_classifier(X_train_features, y_new, X_test_features, y_test)

        y_new = y_new.values.reshape(-1, 1)
        y_test = y_test.values.reshape(-1, 1)

        history = self.train_model(X_new, y_new, X_test, y_test, epochs)
        return history, accuracy

hidden_units = 256
dropout_rate = 0.7
input_shape = (126, 1)
num_classes = 2
learning_rate = 0.001
batch_size = 64
early_stopping_threshold = 0.00001
alpha = 0.01

# Initialize and compile model
try: del model
except: pass

model = ContAuth(hidden_units, dropout_rate, input_shape, num_classes, learning_rate, batch_size, early_stopping_threshold, alpha)
model.compile_model()

# Print model summary
print(model.model.summary())


In [ ]:
!pip install mlrose-hiive

import mlrose_hiive as mlrose
import numpy as np
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score


class ContAuth:
    def __init__(self, hidden_units, dropout_rate, input_shape, num_classes, learning_rate, batch_size, early_stopping_threshold, alpha):
        self.hidden_units = hidden_units
        self.dropout_rate = dropout_rate
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.early_stopping_threshold = early_stopping_threshold
        self.alpha = alpha
        self.model = self.create_model()

    def create_model(self):
        inputs = Input(shape=self.input_shape)
        x = LSTM(self.hidden_units, return_sequences=False)(inputs)
        x = Dropout(self.dropout_rate)(x)
        x = Dense(self.hidden_units, activation='relu')(x)
        x = Dropout(self.dropout_rate)(x)
        outputs = Dense(self.num_classes, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=outputs)
        return model

    def compile_model(self):
        optimizer = Adam(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer,
                           loss='sparse_categorical_crossentropy',
                           metrics=['accuracy'])

    def train_model(self, X_train, y_train, X_val, y_val, epochs):
        if epochs > 20:
            pat = 20
        else:
            pat = 2

        early_stopping = EarlyStopping(monitor='val_loss',
                                       patience=pat,
                                       min_delta=self.early_stopping_threshold,
                                       restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                      patience=2, min_lr=0.00001, mode='auto', verbose=1)
        checkpointer = ModelCheckpoint("saved_model/model_t2.keras", verbose=1, save_best_only=True)

        history = self.model.fit(X_train, y_train,
                                 validation_data=(X_val, y_val),
                                 epochs=epochs,
                                 batch_size=self.batch_size,
                                 callbacks=[early_stopping, reduce_lr, checkpointer])

        return history

    def extract_features(self, X):
        feature_extractor = Model(inputs=self.model.input, outputs=self.model.layers[-1].output)
        features = feature_extractor.predict(X)
        return features

    def train_sgd_classifier(self, X_train_features, y_train, X_val_features, y_val):
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_features)
        X_val_scaled = scaler.transform(X_val_features)

        sgd = SGDClassifier(loss='hinge', penalty='elasticnet', alpha=self.alpha, l1_ratio=0.5, max_iter=1000, tol=1e-3)
        sgd.fit(X_train_scaled, y_train)

        y_val_pred = sgd.predict(X_val_scaled)
        accuracy = accuracy_score(y_val, y_val_pred)
        return sgd, accuracy

    def data_incremental_training(self, X_new, y_new, X_test, y_test, epochs):
        X_train_features = self.extract_features(X_new)
        X_test_features = self.extract_features(X_test)

        sgd, accuracy = self.train_sgd_classifier(X_train_features, y_new, X_test_features, y_test)

        y_new = y_new.values.reshape(-1, 1)
        y_test = y_test.values.reshape(-1, 1)

        history = self.train_model(X_new, y_new, X_test, y_test, epochs)
        return history, accuracy


def fitness_function(params):
    # Map params to the real hyperparameter ranges
    hidden_units = int(params[0] * (512 - 64) + 64)  # Map 0.0-1.0 to 64-512
    dropout_rate = params[1] * (0.7 - 0.2) + 0.2  # Map 0.0-1.0 to 0.2-0.7
    learning_rate = params[2] * (0.01 - 0.0001) + 0.0001  # Map 0.0-1.0 to 0.0001-0.01
    batch_size = int(params[3] * (128 - 16) + 16)  # Map 0.0-1.0 to 16-128

    cont_auth_model = ContAuth(hidden_units=hidden_units,
                               dropout_rate=dropout_rate,
                               input_shape=(X_train.shape[1], X_train.shape[2]),
                               num_classes=2,
                               learning_rate=learning_rate,
                               batch_size=batch_size,
                               early_stopping_threshold=0.001,
                               alpha=0.01)

    cont_auth_model.compile_model()

    history = cont_auth_model.train_model(X_train, y_train, X_val, y_val, epochs=5)

    val_accuracy = max(history.history['val_accuracy'])

    return -val_accuracy


problem = mlrose.ContinuousOpt(length=4,  # 4 hyperparameters to optimize
                               fitness_fn=mlrose.CustomFitness(fitness_function),
                               maximize=True,
                               min_val=0.0,  # Minimum value for all parameters
                               max_val=1.0,  # Maximum value for all parameters
                               step=0.25)

best_params, best_fitness = mlrose.genetic_alg(problem, max_iters=5, random_state=42)

best_hidden_units = int(best_params[0])
best_dropout_rate = best_params[1]
best_learning_rate = best_params[2]
best_batch_size = int(best_params[3])

print(f"Best Hidden Units: {best_hidden_units}")
print(f"Best Dropout Rate: {best_dropout_rate}")
print(f"Best Learning Rate: {best_learning_rate}")
print(f"Best Batch Size: {best_batch_size}")

final_model = ContAuth(hidden_units=best_hidden_units,
                       dropout_rate=best_dropout_rate,
                       input_shape=(X_train.shape[1], X_train.shape[2]),
                       num_classes=2,
                       learning_rate=best_learning_rate,
                       batch_size=best_batch_size,
                       early_stopping_threshold=0.001,
                       alpha=0.01)

final_model.compile_model()

final_history = final_model.train_model(X_train, y_train, X_test, y_test, epochs=50)

final_loss, final_accuracy = final_model.model.evaluate(X_test, y_test)
print(f"Final Test Accuracy: {final_accuracy}")


In [ ]:
#Initial training
# After running one of above three

history = model.train_model(X_train, y_train, X_test, y_test, 100)

In [ ]:
#Data incremental learning

#New data is X_val, y_val

X_val_train, X_val_test, y_val_train, y_val_test = train_test_split(X_val, y_val, test_size=0.2, random_state=42)
history1, accuracy = model.data_incremental_training(X_val_train, y_val_train, X_val_test, y_val_test, 10)
print("Accuracy: ", accuracy)

In [ ]:

model_loaded = model.model

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve
predictions = model_loaded.predict(X_test)

# print(len(predictions))
# print(predictions)

for sigmoid_threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
  labels = [0 if pred > sigmoid_threshold else 1 for pred in predictions[:, 0]]

  conf_matrix = confusion_matrix(y_test, labels)

  TN, FP, FN, TP = conf_matrix.ravel()

  FAR = FP / (FP + TN)
  FRR = FN / (FN + TP)

  print(f"Sigmoid Threshold: {sigmoid_threshold}")
  print(f"False Acceptance Rate (FAR): {FAR}")
  print(f"False Rejection Rate (FRR): {FRR}")

  fpr, tpr, thresholds = roc_curve(y_test, predictions[:, 1])

  frr = 1 - tpr

  eer_threshold = thresholds[np.nanargmin(np.abs(frr - fpr))]
  eer = fpr[np.nanargmin(np.abs(frr - fpr))]

  # print(f"EER Threshold: {eer_threshold}")
  print(f"Equal Error Rate (EER): {eer}")
  print()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig("Loss_LSTM.pdf", format="pdf", bbox_inches="tight")
plt.show()